In [7]:
# ============================================================
# Imports
# ============================================================

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")

# ============================================================
# Utility Functions
# ============================================================

def log_return(series):
    return np.log(series).diff()

def safe_diff(series):
    return series.diff()

# ============================================================
# Data Loader
# ============================================================

def load_excel_series(path, sheet, column):
    df = pd.read_excel(path, sheet_name=sheet)
    df["Date"] = pd.to_datetime(df["Date"])
    return df.set_index("Date")[column].sort_index()

# ============================================================
# Load Data
# ============================================================

portfolio = load_excel_series("Workshop Data.xlsx", "Portfolio", "TLT Position")
tlt_div   = load_excel_series("Workshop Data.xlsx", "Portfolio", "TLT Dividends")

tlt_vol = load_excel_series("Workshop Data.xlsx", "TLT", "Volume")

vix = load_excel_series("MOVE Vix prices.xlsx", "VIX", "PX_LAST")

ust_tr_index = load_excel_series(
    "Indexes and Spreads Data 01.09.xlsx",
    "UST Index",
    "PX_LAST"
)

ust10_yield = load_excel_series(
    "Indexes and Spreads Data 01.09.xlsx",
    "10yUST Yields",
    "PX_LAST"
)

ig_index = load_excel_series(
    "Indexes and Spreads Data 01.09.xlsx",
    "IG Index",
    "OAS_SOVEREIGN_CURVE"
)

hy_index = load_excel_series(
    "Indexes and Spreads Data 01.09.xlsx",
    "HY Index",
    "OAS_SOVEREIGN_CURVE"
)

# ============================================================
# Construct TLT Total Return
# ============================================================

tlt_total_return = (
    portfolio + tlt_div[::-1].cumsum()[::-1]
)

# ============================================================
# Model Function
# ============================================================

def run_tlt_factor_model(freq_label, freq):

    print(f"\n================ {freq_label.upper()} TLT MODEL =================")

    # ---------------- TLT Return ----------------
    tlt_ret = (
        tlt_total_return
        .resample(freq)
        .last()
        .pct_change()
        .rename("TLT_Return")
    )

    # ---------------- Treasury Market Factor ----------------
    ust_tr = (
        ust_tr_index
        .resample(freq)
        .last()
    )
    ust_tr_ret = log_return(ust_tr).rename("UST_Total_Return")

    # ---------------- Rate Duration Shock ----------------
    ust10_rs = ust10_yield.resample(freq).last()
    d_ust10 = safe_diff(ust10_rs).rename("Delta_10Y")

    # ---------------- Volatility ----------------
    vix_rs = vix.resample(freq).last()
    vix_change = safe_diff(vix_rs).rename("VIX_Change")

    # ---------------- Credit Stress ----------------
    ig_rs = ig_index.resample(freq).last()
    hy_rs = hy_index.resample(freq).last()
    credit_stress = (safe_diff(hy_rs) - safe_diff(ig_rs)).rename("Credit_Stress")

    # ---------------- Liquidity ----------------
    tlt_vol_rs = tlt_vol.resample(freq).last()
    liquidity = log_return(tlt_vol_rs).rename("Liquidity")

    # ---------------- Assemble ----------------
    factors = pd.concat(
        [
            ust_tr_ret,
            d_ust10,
            vix_change,
            credit_stress,
            liquidity
        ],
        axis=1
    )

    data = pd.concat([tlt_ret, factors], axis=1, join="inner").dropna()

    y = data["TLT_Return"]
    X = data.drop(columns="TLT_Return")

    # ========================================================
    # OLS
    # ========================================================

    X_ols = sm.add_constant(X)
    ols = sm.OLS(y, X_ols).fit()

    print("\nOLS Summary:")
    print(ols.summary())

    # ========================================================
    # PCA
    # ========================================================

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    pca = PCA(n_components=min(4, X.shape[1]))
    pcs = pca.fit_transform(X_scaled)

    pc_df = pd.DataFrame(
        pcs,
        index=X.index,
        columns=[f"PC{i+1}" for i in range(pcs.shape[1])]
    )

    explained = pd.Series(
        pca.explained_variance_ratio_,
        index=pc_df.columns,
        name="Explained Variance"
    )

    print("\nPCA Explained Variance:")
    print(explained)

    pca_model = sm.OLS(y, sm.add_constant(pc_df)).fit()

    print("\nPCA Regression Summary:")
    print(pca_model.summary())

    return {
        "OLS": ols,
        "PCA": pca_model,
        "Loadings": pd.DataFrame(
            pca.components_.T,
            index=X.columns,
            columns=pc_df.columns
        ),
        "Explained_Variance": explained
    }

# ============================================================
# Run Frequencies
# ============================================================

daily   = run_tlt_factor_model("Daily", "D")
weekly  = run_tlt_factor_model("Weekly", "W")
monthly = run_tlt_factor_model("Monthly", "M")



================ DAILY TLT MODEL =================

OLS Summary:
                            OLS Regression Results                            
Dep. Variable:             TLT_Return   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     2.329
Date:                Fri, 16 Jan 2026   Prob (F-statistic):             0.0408
Time:                        17:53:28   Log-Likelihood:                 3169.6
No. Observations:                 971   AIC:                            -6327.
Df Residuals:                     965   BIC:                            -6298.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------